In [2]:
!pip install mediapipe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 20.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.16.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.5 which is incompatible.


In [10]:
import csv
import cv2
import mediapipe as mp

mp_pose = mp.solutions.pose

action_label = "sentarse"

video_path = 'sentarseFrente.mp4'
cap = cv2.VideoCapture(video_path)

video_name = video_path.split('/')[-1]

frame_count = 0

with open('sentarseFrente.csv', mode='w', newline='') as file:
    writer = csv.writer(file)

    header = ['label', 'frame_number', 'timestamp', 'video_name']
    for i in range(33):
        header += [f'x{i}', f'y{i}', f'z{i}']
    writer.writerow(header)

    with mp_pose.Pose() as pose:
        while cap.isOpened():
            ret, frame = cap.read()

            if not ret:
                break

            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            results = pose.process(rgb_frame)

            if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark

                timestamp = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000
                row = [action_label, frame_count, timestamp, video_name]

                for landmark in landmarks:
                    row.append(landmark.x)
                    row.append(landmark.y)
                    row.append(landmark.z)

                writer.writerow(row)

            frame_count += 1

cap.release()
cv2.destroyAllWindows()
